In [3]:
# import necessary libraries
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import inspect, create_engine, func, MetaData, Table
import pandas as pd
import numpy as np
import json

In [4]:
# Database Setup
engine = create_engine('sqlite:///SevereWeather.sqlite', echo=False)

# produce our own MetaData object
metadata = MetaData()

# we can reflect it ourselves from a database, using options
# such as 'only' to limit what tables we look at...
metadata.reflect(engine)

Base = automap_base()
Base.prepare(engine, reflect=True)

In [5]:
inspector = inspect(engine)
Events = Table('Events',metadata)
inspector.reflecttable(Events, None)
session = Session(bind=engine)

In [6]:
events_df = pd.read_sql_table('Events', engine)

In [9]:
#events_df.info()

In [11]:
tornado_df = events_df.loc[events_df['type']=="torn"]

In [12]:
value = ['2010']
tornado_2010_df = tornado_df[tornado_df.yr.isin(value)]

,level_0,index,yr,mo,dy,st,mag,inj,fat,loss,closs,slat,slon,type,date_time
0,0,0,2010,1,18,CA,0.0,0,0,0.00,0.0,36.7300,-119.8600,torn,2010-01-18 17:24:00
1,1,1,2010,1,19,CA,0.0,0,0,0.00,0.0,34.4248,-119.8770,torn,2010-01-19 12:32:00
2,2,2,2010,1,19,CA,1.0,0,0,0.50,0.0,33.7160,-118.1250,torn,2010-01-19 14:55:00
3,3,3,2010,1,20,LA,0.0,0,0,0.00,0.0,30.8150,-92.0455,torn,2010-01-20 15:27:00
4,4,4,2010,1,20,TX,1.0,0,0,0.04,0.0,31.5095,-93.9232,torn,2010-01-20 16:26:00


In [14]:
#tornado_2010_df.info()

In [15]:
locations = tornado_2010_df[['slat', 'slon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[1:5]

[[34.4248, -119.87700000000001],
 [33.716, -118.125],
 [30.815, -92.0455],
 [31.5095, -93.9232]]

In [16]:
from geojson import Feature, FeatureCollection, Point

In [19]:
# columns used for constructing geojson object
features = tornado_2010_df.apply(
    lambda row: Feature(geometry=Point((float(row['slon']), float(row['slat'])))),
    axis=1).values.tolist()

# all the other columns used as properties
properties = tornado_2010_df.drop(['slat', 'slon'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [20]:
with open('tornado_2010.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

TypeError: Object of type 'Timestamp' is not JSON serializable